# Converting NDVI Values to Admin 1 and Admin 2

In [1]:
%load_ext jupyter_black
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats

In [2]:
data_dir = Path(os.getenv("HAM_DIR")) / "vegetation/ndvi/"
output_dir = Path(os.getenv("HAM_DIR")) / "vegetation/"
shp_dir = Path(os.getenv("AA_DATA_DIR")) / "public/raw/ner/cod_ab/ner_cod_ab/"

In [3]:
ner_shp_adm0 = gpd.read_file(shp_dir / "adm0.shp.zip")
ner_shp_adm1 = gpd.read_file(shp_dir / "adm1.shp.zip")
ner_shp_adm2 = gpd.read_file(shp_dir / "adm2.shp.zip")

In [4]:
ner_shp_adm0.total_bounds

array([ 0.16625  , 11.69697  , 15.99564  , 23.5331862])

## Admin 1

In [5]:
ner_shp_adm1.drop(
    ["OBJECTID", "Shape_Leng", "Shape_Area", "ISO3", "ISO2"], axis=1, inplace=True
)
ner_shp_adm1

,adm_01,rowcacode1,NOMREG,geometry
0,Agadez,NER001,AGADEZ,"POLYGON ((11.98901 23.53319, 11.99796 23.51602..."
1,Diffa,NER002,DIFFA,"POLYGON ((15.55942 18.00625, 15.55675 17.95649..."
2,Dosso,NER003,DOSSO,"POLYGON ((3.66840 11.98401, 3.66388 11.98413, ..."
3,Maradi,NER004,MARADI,"POLYGON ((7.76947 14.42108, 7.82971 14.41669, ..."
4,Niamey,NER008,NIAMEY,"POLYGON ((2.09448 13.61969, 2.09711 13.61829, ..."
5,Tahoua,NER005,TAHOUA,"POLYGON ((4.87129 18.67932, 4.83798 18.58273, ..."
6,Tillabéri,NER006,TILLABERI,"POLYGON ((3.88129 15.68909, 4.01208 15.39410, ..."
7,Zinder,NER007,ZINDER,"POLYGON ((12.00488 17.41852, 12.00427 15.99390..."


In [6]:
file_list = os.listdir(data_dir)

adm1_ndvi_df = pd.DataFrame()
for file in file_list:
    file_path = Path(data_dir / file)
    input_raster = rasterio.open(file_path)
    array = input_raster.read(1)
    summary_stats = zonal_stats(
        ner_shp_adm1,
        array,
        stats=["mean", "median"],
        nodata=0,
        all_touched=True,
        affine=input_raster.transform,
    )
    adm1_stats = pd.DataFrame(summary_stats)
    file_df = pd.merge(
        ner_shp_adm1[["rowcacode1", "NOMREG"]],
        adm1_stats,
        left_index=True,
        right_index=True,
    )
    # file_df["date"] = file[(file.find("data") + 4) : file.find(".tiff")]
    adm1_ndvi_df = pd.concat([adm1_ndvi_df, file_df], axis=0)

KeyboardInterrupt: 

In [7]:
adm1_ndvi_df

,rowcacode1,NOMREG,mean,median,date
0,NER001,AGADEZ,112.010722,112.0,207.ti
1,NER002,DIFFA,112.994590,111.0,207.ti
2,NER003,DOSSO,121.716683,121.0,207.ti
3,NER004,MARADI,120.451096,120.0,207.ti
4,NER008,NIAMEY,117.845657,117.0,207.ti
...,...,...,...,...,...
3,NER004,MARADI,117.664521,117.0,231.ti
4,NER008,NIAMEY,114.940038,114.0,231.ti
5,NER005,TAHOUA,113.829265,114.0,231.ti
6,NER006,TILLABERI,114.630654,113.0,231.ti


In [ ]:
adm1_ndvi_df.to_csv(output_dir / "ner_adm1_ndvi.csv", index=False)

## Admin 2

In [ ]:
ner_shp_adm2.drop(
    ["OBJECTID", "Shape_Leng", "Shape_Area", "adm_01", "rowcacode1", "ISO3", "ISO2"],
    axis=1,
    inplace=True,
)
ner_shp_adm2

In [ ]:
adm2_ndvi_df = pd.DataFrame()
for file in file_list:
    file_path = Path(data_dir / file)
    input_raster = rasterio.open(file_path)
    array = input_raster.read(1)
    summary_stats = zonal_stats(
        ner_shp_adm2,
        array,
        stats=["mean", "median"],
        nodata=0,
        all_touched=True,
        affine=input_raster.transform,
    )
    adm2_stats = pd.DataFrame(summary_stats)
    file_df = pd.merge(
        ner_shp_adm2[["rowcacode2", "NOMDEP"]],
        adm2_stats,
        left_index=True,
        right_index=True,
    )
    file_df["date"] = file[(file.find("data") + 4) : file.find(".tiff")]
    adm2_ndvi_df = pd.concat([adm2_ndvi_df, file_df], axis=0)

In [ ]:
adm2_ndvi_df

In [ ]:
adm2_ndvi_df.to_csv(output_dir / "ner_adm2_ndvi.csv", index=False)